I haven't been able to get Jupyter notebooks working yet with the USGS HPCs, but this cluster can't handle the GBIF data.  So I'm trying this manually in Denali, on a single node, to see if I can make progress.

### Import modules

In [5]:
import fsspec
import dask.dataframe as dd
import intake
import os
import sys
import warnings
from dask.distributed import LocalCluster, Client
warnings.filterwarnings('ignore')

#### Start as Dask client

In [ ]:
#On Denali
#from dask.distributed import LocalCluster, Client
#cluster = LocalCluster(threads_per_worker=1)
#client = Client(cluster)
 
#This didn't work.  It spun up the cluster but crashed on the first command.  Let's try it without the cluster.

## Explore dask/data structure

Following tips and tutorials from these:
- https://www.youtube.com/watch?v=6EozEvfTcBI
- https://www.youtube.com/watch?v=LKIRAzsqLb0

In [ ]:
#Working locally in Denali, start with subset of parquet, first partition.

df = dd.read_parquet('/caldera/projects/css/sas/bio/spec_obs/gbif/snapshot/occurrence.parquet/000000') #first 10 = 00000*

In [35]:
df.head()

,gbifid,datasetkey,occurrenceid,kingdom,phylum,class,order,family,genus,species,...,identifiedby,dateidentified,license,rightsholder,recordedby,typestatus,establishmentmeans,lastinterpreted,mediatype,issue
0,3534344612,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1178111150,Animalia,Chordata,Aves,Passeriformes,Cardinalidae,Passerina,Passerina ciris,...,[],NaT,CC_BY_4_0,None,[obsr455843],[],None,2022-09-08 14:35:29.672,[],[]
1,3534604870,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD_CAN:OBS1179048592,Animalia,Chordata,Aves,Passeriformes,Emberizidae,Melospiza,Melospiza lincolnii,...,[],NaT,CC_BY_4_0,None,[obsr232608],[],None,2022-09-08 14:35:31.349,[],[]
2,3534885128,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1180074221,Animalia,Chordata,Aves,Passeriformes,Parulidae,Setophaga,Setophaga petechia,...,[],NaT,CC_BY_4_0,None,[obsr313985],[],None,2022-09-08 14:35:34.634,[],[]
3,3535431386,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1180952857,Animalia,Chordata,Aves,Psittaciformes,Psittacidae,Pionus,Pionus menstruus,...,[],NaT,CC_BY_4_0,None,[obsr754360],[],None,2022-09-08 14:35:37.436,[],[]
4,3535679644,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1181682586,Animalia,Chordata,Aves,Piciformes,Picidae,Melanerpes,Melanerpes carolinus,...,[],NaT,CC_BY_4_0,None,[obsr14802],[],None,2022-09-08 14:35:39.348,[],[]


In [36]:
df.dtypes

## Some thoughts about the data

- Many columns won't be needed.  What's needed are:
    - kingdom
    - class
    - species
    - countrycode
    - decimallatitude
    - decimallongitude
    - year

- decimallatitude and decimal longitude can be converted to int32 to save memory since (I think) most coordinates aren't accurate beyond 4 or 5 decimal places.

Let's check a partition size (in MB). I think it should be aroudn 85 MB, since there are 2021 partitions and the parquet file is ~ 173 GB.


In [14]:
#a single partition
#df.partitions[0].memory_usage(deep=True).compute()/1e6

In [ ]:
df.partitions[0].memory_usage(deep=True).sum().compute()/1e6

Hmm, that returns 4014 MB. If I up it to the first 10 partitions it remains the same, which makes sense.

In [42]:
part_list = df.memory_usage_per_partition(deep=True).compute() / 1e6

Right, there is only one partition because that's what I read in. Now I understand each file within a parquet is read in as a partition. From some quick browsing though, it doesn't look like they're chunked by anything obvious. The above kicked out an error with 10 partitions.  Maybe it is too much?

Let me try repartitioning to 80, for the 80 cores on my single node.

In [ ]:
df1 = df.repartition(80)

That makes them each about 50MB, that's pretty nice. When upped to 10 partitions the first one was 509 MB. After repartitioning it still couldn't handle the usage per partition function. But it should be ~ 80* 500 MB = 40 GB

In [45]:
 part_sum = part_list.sum()
part_sum #So 4 GB in total. 

4762.149198174

## More thoughts...

- not clear if any of these are horrible, but from the tutorial, it's likely that the objects should be converted to "string[pyarrow]"  Not too suprisingly, these are the most expensive.
- After thinking about it some more, it's probably not safe to convert float 64 to float32. Too much precision would be lost.
- Let's check total memory volume 

In [15]:
import dask

dask.utils.format_bytes(
   df1.memory_usage(deep=True).compute().sum()
)

_This says it's 3.74 GB_

In [53]:
conversions = {}
for column, dtype in df.dtypes.items():
    if dtype == "object":
        conversions[column] = "string[pyarrow]"
        
conversions

{'datasetkey': 'string[pyarrow]',
 'occurrenceid': 'string[pyarrow]',
 'kingdom': 'string[pyarrow]',
 'phylum': 'string[pyarrow]',
 'class': 'string[pyarrow]',
 'order': 'string[pyarrow]',
 'family': 'string[pyarrow]',
 'genus': 'string[pyarrow]',
 'species': 'string[pyarrow]',
 'infraspecificepithet': 'string[pyarrow]',
 'taxonrank': 'string[pyarrow]',
 'scientificname': 'string[pyarrow]',
 'verbatimscientificname': 'string[pyarrow]',
 'verbatimscientificnameauthorship': 'string[pyarrow]',
 'countrycode': 'string[pyarrow]',
 'locality': 'string[pyarrow]',
 'stateprovince': 'string[pyarrow]',
 'occurrencestatus': 'string[pyarrow]',
 'publishingorgkey': 'string[pyarrow]',
 'basisofrecord': 'string[pyarrow]',
 'institutioncode': 'string[pyarrow]',
 'collectioncode': 'string[pyarrow]',
 'catalognumber': 'string[pyarrow]',
 'recordnumber': 'string[pyarrow]',
 'identifiedby': 'string[pyarrow]',
 'license': 'string[pyarrow]',
 'rightsholder': 'string[pyarrow]',
 'recordedby': 'string[pyarrow

In [54]:
df1 = df1.astype(conversions)

**How big is it now?**

In [55]:
dask.utils.format_bytes(
   df1.memory_usage(deep=True).compute().sum()
)

Wow, that was significant for the strings. Now we're looking at 0.99 GB. I couldn't get a number on volume before or after conversion.

## Further filtering...

From this data quality presentation, there are some recommendations about flags and filters to use.

- zero-zero coordinates should be removed (although if we're just thinking about USA this isn't a problem)
- country coordinate mismatch (under issues) can be used to filter to USA, and otherwise remove spurious data.
- remove absence records
- remove coordinate invalid records
- remove coordinate out of range records
- remove fossil records
- remove living specimen records (in zoo, botanical gardens)
- count and (maybe) remove data in country centroids

*Select the columns we need for filtering. First examine values*

Some memory is saved by converting issue, basisofrecord, kingdom, and countrycode to category...but according to Ayush, there's a known bug with categorical columns when using to_parquet to save the dask dataframe into parquet files. He says, "It affects some environments (at least it did mine) while transforming the encoding."  So we'll avoid back convert these when writing to parquet.

In [57]:
#df['issue'] = df.issue.astype('category')
#df['basisofrecord'] = df.basisofrecord.astype('category')
#df['countrycode'] = df.countrycode.astype('category')
#df['kingdom'] = df.kingdom.astype('category')

#df.partitions[0].memory_usage(deep=True).compute()/1000000

## Find levels

Because dask [splits categoricals](https://docs.dask.org/en/stable/dataframe-categoricals.html), if we want to know all possible levels, we need to make them "known" first...but this requires scan of the entire data, so not really worth it.  I got the levels by going to GBIF.org and looking at the URL after filtering these with the search tool. Include gbifid as an index for groupby

In [58]:
#df = df.categorize(columns= ['issue', 'basisofrecord'])

In [ ]:
df1 = df[["gbifid",
          "issue",
         "basisofrecord",
         "occurrencestatus",
         "kingdom",
         "class",
         "countrycode",
         "decimallongitude",
         "decimallatitude",
         "year",
         "species"]]

In [64]:
df1 = df1[~df1['issue'].isin(["['COUNTRY_COORDINATE_MISMATCH']",
                             "['RECORDED_DATE_INVALID']",
                             "['COORDINATE_INVALID']"])
         ]

In [65]:
df1 = df1[df1['occurrencestatus'] == "PRESENT"]

In [66]:
df1 = df1[df1['basisofrecord'].isin(["OBSERVATION",
                                  "MACHINE_OBSERVATION",
                                  "HUMAN_OBSERVATION",
                                  "MATERIAL_SAMPLE",
                                  "PRESERVED_SPECIMEN",
                                  "OCCURRENCE"])
       ]

Check size again

In [ ]:
dask.utils.format_bytes(
   df1.memory_usage(deep=True).compute().sum()
)

Dang, down to 167.2 MB. For 10 partitions it was 9.36 GB

Let's subset to USA. I learned here that it's necessary to use double quotes around the US.

In [67]:
df1 = df1[df1.countrycode == "US"]

filter to the 2010s

In [68]:
df1 = df1[df1.year >= 2010]
df1 = df1[df1.year <= 2019]

Remove unneeded columns

In [69]:
df1 = df1[["gbifid",
           "kingdom",
         "class",
         "decimallongitude",
         "decimallatitude",
         "year",
         "species"]]

In [70]:
df1.head(n = 2)

,kingdom,class,decimallongitude,decimallatitude,year,species
170,Animalia,Aves,-89.28571,38.666687,2010,Buteo jamaicensis
171,Animalia,Aves,-155.32988,19.644060,2010,Lophura leucomelanos


Check size again

In [ ]:
dask.utils.format_bytes(
   df1.memory_usage(deep=True).compute().sum()
)

23 MB. For 10 partitions we're down to 182 MB. For 100 partitions we're down to 1.66 GB (but it took a long time to compute that)

Drop NA values in species (not totally sure we should drop these as much as aggregate them at genus, but that's a discussion for another day.) and decimallatitude (could use longtitude instead). _Follow up: I think this dropna is really computationally expensive (from watching the workers) but I'm not sure why.  Something to research._

In [ ]:
df2 = df1.persist()

Took a minute or so.

_It may have helped a little.  There is still a fair amount on disk (in grey) but I think that's going to have to be the way it is because of the size of the cluster.  Check volume below._

In [ ]:
#On Denali, write parquet, just to see.  Keep in mind what Anush said about categoricals.

dd.to_parquet(df2, '/caldera/projects/css/sas/bio/spec_obs/gbif/snapshot/US_2010s_occurrence.parquet/000000')

Wow extremely quick (like a second).  What about without persist? Definitely not as fast, and that makes sense. Probably took 30 -60 seconds.

In [ ]:
dd.to_parquet(df1, '/caldera/projects/css/sas/bio/spec_obs/gbif/snapshot/US_2010s_occurrence.parquet/000000')

Ok, let's get down to what we actually need...

In [ ]:
df2.groupby(["year", "decimallongitude","decimallatitude"]).count().persist()  #tooks about 1 min

But returned something unexpected:

                                        kingdom  class  species
year   decimallongitude decimallatitude
2010.0 -171.74380       63.77877               1      1        1
       -171.72145       63.77930               2      2        2
       -170.29675       57.17012               1      1        1
       -169.60693       56.57764               1      1        1
       -168.11691       65.63562               2      2        2
>>>

I want kingdom, class and species names in there.

In [33]:
df2 = df1.groupby(["year", "decimallongitude","decimallatitude", "kingdom", "class", "species"]).count().persist()  #tooks about 1 min

Which gives me:

Empty DataFrame
Columns: []
Index: [(2010.0, -171.7438, 63.77877, Animalia, Aves, Corvus corax), (2010.0, -171.72145, 63.7793, Animalia, Aves, Fratercula corniculata), (2010.0, -171.72145, 63.7793, Animalia, Aves, Polysticta stelleri), (2010.0, -170.29675, 57.17012, Animalia, Aves, Arenaria interpres), (2010.0, -169.60693, 56.57764, Animalia, Aves, Aethia cristatella)]

In [ ]:
df1.groupby("year").count().persist()  #tooks about 1 min

Interesting that persist doesn't need to be on the end of count if it was called on the whole df.  But the result of count won't print unless it's assigned to an object and you execute compute or persist.

df3 = df2.groupby(["year", "decimallongitude","decimallatitude", "kingdom", "class", "species"]).count()

In [ ]:
Q = df2.groupby(["year", "decimallongitude","decimallatitude"]).count()

Anyway, its the order of the grouping that was wrong.

In [ ]:
 Q = df2.groupby(["species", "year", "decimallongitude","decimallatitude"]).count()

                                                              kingdom  class
species              year   decimallongitude decimallatitude
Abortiporus biennis  2018.0 -83.665780       42.809880              1      1
Abutilon theophrasti 2012.0 -86.212167       42.923222              1      1
Acalypha virginica   2014.0 -83.432850       42.195940              1      1
                     2016.0 -85.970167       42.607861              1      1
                     2018.0 -86.135517       43.854500              1      1


Ok, so Anush had suggested a good answer.  Keep gbifid in as an index since dask doesn't use the 'as_index=false; argument like pandas does. Also see this problem with https://examples.dask.org/dataframes/02-groupby.html#Many-groups it allows you to write out to many partitions.

In [ ]:
df3 = df2.groupby(["year", "decimallongitude","decimallatitude", "kingdom", "class", "species"], group_keys=False).count().compute()

Only takes a minute, even with 10 partitions (621835 rows). Not much longer with 100 partitions (4,806,414 rows) Group keys doesn't seem to do anything.  What does it look like if I write it to csv? need to reset index or you only get the gbifid column

In [ ]:
df3.reset_index().to_csv('/caldera/projects/css/sas/bio/spec_obs/gbif/snapshot/US_2010s_occurrence.parquet/test_write_csv.csv', index=False)

## Mess around with datashader

followed this: https://www.youtube.com/watch?v=LKIRAzsqLb0

In [ ]:

# import datashader
# from datashader import transfer_functions as tf
# from datashader.colors import Hot
# import holoviews as hv

# def render(df1, x_range=(-125, -65), y_range=(24, 50)):
#     # Plot
#     canvas = datashader.Canvas(
#         x_range=x_range,
#         y_range=y_range,
#     )
#     agg = canvas.points(
#         source=df, 
#         x="decimallongitude", 
#         y="decimallatitude", 
#         agg=datashader.count("kingdom"),
#     )
#     return datashader.transfer_functions.shade(agg, cmap=Hot, how="eq_hist")

In [ ]:
#render(df1)

## List your clusters, then shut them all down

In [5]:
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()

In [6]:
gateway.list_clusters()

[]

In [3]:
for cluster in gateway.list_clusters():
    gateway.stop_cluster(cluster_name=cluster.name)
    print(f'stopped {cluster.name}')

stopped dev.ba22da0d08f74fd99f2884edd0040909


In [4]:
gateway.list_clusters()

[]